### Loading the super-tensor

Testing the class __super_tensor__ to load the super-tensor, and compute the cortical temporal networks.

In [1]:
import numpy as np
from GDa.super_tensor import super_tensor

In [2]:
freqs = np.arange(4,60,2)
delta = 15

st = super_tensor(raw_path = 'GrayLab/', monkey = 'lucy', 
                  date = '150128', session = 1, delta = delta, freqs = freqs)

In [3]:
import time
start = time.time()
st.load_super_tensor(use_gpu = True)
end = time.time()
print(end - start)

/home/vinicius/GrayData-Analysis/GDa/super_tensor.py:48: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "load_super_tensor_gpu" failed type inference due to: Untyped global name 'self': cannot determine Numba type of <class 'GDa.super_tensor.super_tensor'>

File "GDa/super_tensor.py", line 50:
			def load_super_tensor_gpu():
				_super_tensor = np.zeros([self.nT, self.nP, self.freqs.shape[0], self.tarray.shape[0]],dtype=np.float32)
    ^

  @vectorize([float32()])
/home/vinicius/.local/lib/python3.6/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "load_super_tensor_gpu" was compiled in object mode without forceobj=True.

File "GDa/super_tensor.py", line 49:
			@vectorize([float32()])
			def load_super_tensor_gpu():
   ^

  state.func_ir.loc))
/home/vinicius/.local/lib/python3.6/site-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object m

ValueError: At least one iterator operand must be non-NULL

In [ ]:
np.float32